# Ejercicio . Sistema Lotka-Volterra.

Las ecuaciones de Lotka-Volterra, también son conocidas como ecuaciones de depredador-presa. Está descrito por un sistema de 2 ecuaciones diferenciales no lineales de primer orden.

Se utiliza frecuentemente para describir la dinámica de sistemas biológicos donde interaccionan dos especies: **depredadores** y **presas**.

El sistema evoluciona de acuerdo al par de ecuaciones:

$$
\begin{align*}
\dfrac{du}{dt} &= a \: u - b \: u \: v \\
\dfrac{dv}{dt} &= -c \: v + d \: b\: u\: v
\end{align*}
$$

donde:  
1. $u$ es el número de presas (ej. Conejos)
2. $v$ es el número de depredadores (ej. Zorros)
3. $a$ es la tasa natural de crecimiento de conejos, sin que haya zorros.
4. $b$ es la tasa natural de la muerte de conejos, debido a la depredación.
5. $c$ es la tasa natural de la muerte del zorro, cuando no hay conejos.
6. $d$ es el factor que describe el número de conejos capturados.

## Poblaciones iniciales de predadores y presas.

Vamos a utilizar $X = [u, v]$ para describir el estado de las poblaciones.

In [ ]:
import numpy as np

def dXdt(X, t = 0):
    return np.array([ a * X[0] - b * X[0] * X[1], -c * X[1] + d * b * X[0] * X[1]])

a = 1.
b = 0.1
c = 1.5
d = 0.75

# Población en equilibrio.

Antes de usar <code>odeint</code> para integrar el sistema, veremos de cerca la posición de equilibrio.

El equilibrio ocurre cuando la tasa de crecimiento es igual a $0$, lo que nos da dos puntos fijos:

In [ ]:
Xf0 = np.array([0., 0.])
Xf1 = np.array([ c/(d * b), a/b])
all(dXdt(Xf0) == np.zeros(2) ) and all(dXdt(Xf1) == np.zeros(2))

# Variable temporal y condiciones iniciales.

Para usar la función <code>odeint</code>, hay que definir el parámetro de tiempo $t$, así como las condiciones inciales de la población: $10$ conejos y $5$ zorros, una vez hecho esto, ya podemos hacer uso de la función <code>odeint></code> con los parámetros indicados.

In [ ]:
from scipy.integrate import odeint

t = np.linspace(0, 15,  1000)
              
X0 = np.array([10, 5])         
            
X = odeint(dXdt, X0, t)

# Graficando la solución.

Una vez obtenido el código para la solución del problema, ahora nos corresponde graficar el conjunto de datos obtenido:

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

conejos, zorros = X.T

plt.plot(t, conejos, 'r-', label='Conejos')
plt.plot(t, zorros  , 'b-', label='Zorros')
plt.grid()
plt.legend(loc='best')
plt.xlabel('Tiempo')
plt.ylabel('Población')
plt.title('Evolución de la población de conejos y zorros')
plt.show()

## Interpretando la gráfica.

La gráfica anterior nos da la información sobre el número tanto de conejos como de zorros durante el intervalo de tiempo estudiado, es decir, tenemos una especie de **censo poblacional**.

# Dinámica de la población.

Para ver la dinámica de las poblaciones propiamente, ahora representamos el *espacio fase del sistema*, por lo que tenemos que hacer algunos ajustes en el código que usamos anteriormente.

## Condiciones de equilibrio.

Consideremos las condiciones de equilibrio, es decir, donde la tasa de crecimiento es cero:

**Nota: las siguientes celdas con código quedarán comentadas, con la finalidad de dar una explicación de lo que hace cada parte, al final se presenta de nuevo el código completo en una sola celda.**

In [ ]:
# Xf0 = np.array([0. , 0.])
# Xf1 = np.array([ c/(d * b), a/b])

## Elementos adicionales para la gráfica.

Dibujaremos el espacio fase con algunos elementos visuales con el fin de decoración nada más y obtener una gráfica más atractiva y legible.

## El módulo *color map*.

El módulo <code>cm</code> proporciona un conjunto de mapas de colores predeterminados, así como las funciones necesarias para crear nuevos mapas de color.

Existen varios mapas ya definidos: <code>autumn, bone, cool, copper, flag, gray, hot, hsv, jet, pink, prism, spring, summer, winter, spectral</code>.

In [ ]:
# values = np.linspace(0.3, 0.9, 5)                         

# vcolors = plt.cm.autumn_r(np.linspace(0.3, 1., len(values)))

## Curvas de nivel.

Se van a dibujar ahora las trayectorias para diferentes condiciones iniciales (número de conejos y zorros)

La función <code>zip</code> sirve para reorganizar las listas en python.

Como parámetros admite un conjunto de listas.  Lo que realmente hace es tomar el elemento i-ésimo elemento de cada lista y los une en una tupla, después une todas las tuplas en una lista.

En cada gráfica se modificará el grosor de la línea y el color que se le asocia.

In [ ]:
# for v, col in zip(values, vcolors):
#    X0 = v * Xf1
#    X = odeint(dXdt, X0, t)
#   plt.plot( X[:,0], X[:,1], lw=3.5 * v, color=col, label='({0:1.1f} , {1:1.1f}'.format(X0[0], X0[1]))

## Definición de una malla.

Se define una malla sobre nuestro espacio de solución.

La función <code>meshgrid</code> genera un arreglo n-dimensional para evaluaciones vectoriales de campos n-dimensionales ya sea escalares o vectoriales, a partir de arreglos unidimensionales $x_{1}, x_{2}, \ldots, x_{n}$.

In [ ]:
# ymax = plt.ylim(ymin = 0)[1]
# xmax = plt.xlim(xmin = 0)[1]

# nbpoints = 20

# x = np.linspace(0, xmax, nbpoints)
# y = np.linspace(0, ymax, nbpoints)

# X1, Y1 = np.meshgrid(x, y)

## Calculando la magnitud del vector.

Como vamos a incorporar un vector en el espacio fase para que nos indique la dirección y su magnitud, habrá que calcularlo.

1. Con <code>X1</code> y <code>Y1</code> se crea una malla.
2. Con <code>DX1</code> y <code>DY1</code> se calcula el crecimiento de las poblaciones en la malla.
3. Con la variable <code>M</code> se calcula la norma de la tasa de crecimiento, usando la función <code>np.hypot</code>.
4. La expresión <code>M[M == 0] = 1.</code> evita que tengamos una división entre cero.
5. Con la operación <code>DX1/M</code> y <code>DY1/M</code> se normaliza cada vector.

In [ ]:
# DX1, DY1 = dXdt([X1, Y1])                      

# M = (np.hypot(DX1, DY1))

# M[ M == 0] = 1.                                 

# DX1 /= M                                        
# DY1 /= M

## Dibujando las direcciones del vector.

Se dibujan las direcciones usando <code>quiver</code>, ya que genera el mapa vectorial; requiere de cinco argumentos:
1. Las posiciones <code>X1, Y1</code> de inicio.
2. El valor de las componentes del vector <code>DX1, DY1</code>
3. El color asociado.

El argumento <code>pivot</code> indica en qué parte de la malla se va a colocar el vector.

In [ ]:
# plt.title('Trayectorias y campo de direccion')

# Q = plt.quiver(X1, Y1, DX1, DY1, M, pivot='mid', cmap=plt.cm.jet)

# plt.xlabel('Población de conejos')
# plt.ylabel('Población de zorros')
# plt.legend()
# plt.grid()
# plt.xlim(0, xmax)
# plt.ylim(0, ymax)
# plt.show()

# Código completo.

El siguiente código es el mismo que se explicó en las celdas anteriores, por lo que al ejecutarlo obtendremos el espacio fase del sistema Lotka-Volterra, con los elementos adicionales para una mejor interpretación del mismo.

In [ ]:
Xf0 = np.array([0. , 0.])
Xf1 = np.array([ c/(d * b), a/b])

values  = np.linspace(0.3, 0.9, 5)                         

vcolors = plt.cm.autumn_r(np.linspace(0.3, 1., len(values)))  

plot2 = plt.figure(2)

for v, col in zip(values, vcolors):
    X0 = v * Xf1
    X = odeint(dXdt, X0, t)
    
    plt.plot(X[:,0], X[:,1], lw=3.5 * v, color=col, label='({0:1.1f} , {1:1.1f}'.format(X0[0], X0[1]))

ymax = plt.ylim(ymin = 0)[1]
xmax = plt.xlim(xmin = 0)[1]

nbpoints = 20

x = np.linspace(0, xmax, nbpoints)
y = np.linspace(0, ymax, nbpoints)

X1, Y1  = np.meshgrid(x, y)

DX1, DY1 = dXdt([X1, Y1])                      

M = (np.hypot(DX1, DY1))

M[ M == 0] = 1.                                 

DX1 /= M                                        
DY1 /= M

plt.title('Trayectorias y campo de dirección - Sistema Lotka-Volterra')

Q = plt.quiver(X1, Y1, DX1, DY1, M, pivot='mid', cmap=plt.cm.jet)

plt.xlabel('Población de conejos')
plt.ylabel('Población de zorros')
plt.legend()
plt.grid()
plt.xlim(0, xmax)
plt.ylim(0, ymax)
plt.show()